In [1]:
import azureml
import os
import urllib
import shutil

from azureml.core.model import Model
from azureml.core import Workspace, Run, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import TensorFlow
from azureml.core.image import Image, ContainerImage
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.72


In [5]:
ws = Workspace.create(name='mnistDemo',
         subscription_id='Update Your Subscription Id From azure https://portal.azure.com/#blade/Microsoft_Azure_Billing/SubscriptionsBlade',
         resource_group='AiBootcampDemo',
         location='westeurope',
         exist_ok=True)

In [26]:
ws.get_details()

{'id': '/subscriptions/d113c551-0506-4de2-9561-922d4a545c29/resourceGroups/AiBootcampDemo/providers/Microsoft.MachineLearningServices/workspaces/mnistDemo',
 'name': 'mnistDemo',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': 'bb409e60-c805-4868-a698-49948b869b61',
 'description': '',
 'friendlyName': 'mnistDemo',
 'creationTime': '2019-12-13T12:47:34.3562855+00:00',
 'keyVault': '/subscriptions/d113c551-0506-4de2-9561-922d4a545c29/resourcegroups/aibootcampdemo/providers/microsoft.keyvault/vaults/mnistdemkeyvault52ecda3b',
 'applicationInsights': '/subscriptions/d113c551-0506-4de2-9561-922d4a545c29/resourcegroups/aibootcampdemo/providers/microsoft.insights/components/mnistdeminsightsd97f6233',
 'identityPrincipalId': 'd6f342a9-d739-4253-9db1-476bf458a731',
 'identityTenantId': '39ca0714-93a2-4cdd-afdd-1770e8fa38b7',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/d113c551-0506-4de2-9561-922d4a54

In [7]:
clusterName = 'Jayant'
try:
    computeCluster = ComputeTarget(workspace=ws, name=clusterName)
    print('Found existing compute target "{}"'.format(clusterName))
except ComputeTargetException:
    print('Creating new compute target "{}"...'.format(clusterName))
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NV6', min_nodes=1, max_nodes=2)
    computeCluster = ComputeTarget.create(ws, clusterName, compute_config)
    computeCluster.wait_for_completion(show_output=True)

Creating new compute target "Jayant"...
Creating
Succeeded...............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [8]:
status = computeCluster.get_status()
print('Current node count: ',status.current_node_count)
print('VM Size: ',status.vm_size)
print('Name:',computeCluster.name)
computeCluster.status.node_state_counts.serialize()

Current node count:  1
VM Size:  STANDARD_NV6
Name: Jayant


{'preparingNodeCount': 0,
 'runningNodeCount': 0,
 'idleNodeCount': 1,
 'unusableNodeCount': 0,
 'leavingNodeCount': 0,
 'preemptedNodeCount': 0}

In [9]:
ds = ws.get_default_datastore()
print(ds.name)

workspaceblobstore


In [10]:
os.makedirs('./data/mnist', exist_ok=True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename = './data/mnist/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename = './data/mnist/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename = './data/mnist/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename = './data/mnist/test-labels.gz')

('./data/mnist/test-labels.gz', <http.client.HTTPMessage at 0x7f4bb1af4748>)

In [11]:
ds.upload(src_dir='./data/mnist', target_path='mnist', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/mnist/test-images.gz
Uploading ./data/mnist/test-labels.gz
Uploading ./data/mnist/train-images.gz
Uploading ./data/mnist/train-labels.gz
Uploaded ./data/mnist/train-labels.gz, 1 files out of an estimated total of 4
Uploaded ./data/mnist/test-labels.gz, 2 files out of an estimated total of 4
Uploaded ./data/mnist/test-images.gz, 3 files out of an estimated total of 4
Uploaded ./data/mnist/train-images.gz, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_bad88098752f4c2ebcf5054b707d776d

In [12]:
ExperimentName = 'AiBootcampDemo'

exp = Experiment(workspace=ws, name=ExperimentName)
print(exp.name)

AiBootcampDemo


In [13]:
script_folder = './trainingscripts'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('./train.py', script_folder)
shutil.copy('./utils.py', script_folder)

'./trainingscripts/utils.py'

In [14]:
script_params = {
    '--data-folder': ws.get_default_datastore().as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=computeCluster,
                 entry_script='train.py', 
                 use_gpu=True,
                 framework_version="1.10")

In [15]:
run = exp.submit(config=est)

In [16]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [17]:
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_154ac4b089fdeaa41f2d5b8cb8c08ff56922d731f230c393631fe617ef5ab4c9_d.txt',
 'azureml-logs/65_job_prep-tvmps_154ac4b089fdeaa41f2d5b8cb8c08ff56922d731f230c393631fe617ef5ab4c9_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json']

In [18]:
os.makedirs('model', exist_ok=True)
run.download_file(name='outputs/model/mnist-tf.model.meta', output_file_path='model')
run.download_file(name='outputs/model/mnist-tf.model.index', output_file_path='model')
run.download_file(name='outputs/model/mnist-tf.model.data-00000-of-00001', output_file_path='model')

In [19]:
model = Model.register(ws, model_name='TensorflowMNISTModel', model_path='model', description='MNIST Model')

Registering model TensorflowMNISTModel


In [20]:
print(model.name ,model.id, model.version, sep = '\t')

TensorflowMNISTModel	TensorflowMNISTModel:1	1


In [21]:
%%writefile score.py
import json
import numpy as np
import os
import tensorflow as tf

from azureml.core.model import Model

def init():
    global X, output, sess
    tf.reset_default_graph()
    model_root = Model.get_model_path('TensorflowMNISTModel')
    saver = tf.train.import_meta_graph(os.path.join(model_root, 'mnist-tf.model.meta'))
    X = tf.get_default_graph().get_tensor_by_name("network/X:0")
    output = tf.get_default_graph().get_tensor_by_name("network/output/MatMul:0")
    
    sess = tf.Session()
    saver.restore(sess, os.path.join(model_root, 'mnist-tf.model'))

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    out = output.eval(session = sess, feed_dict = {X: data})
    y_hat = np.argmax(out, axis = 1)
    return json.dumps(y_hat.tolist())

Writing score.py


In [22]:
from azureml.core.runconfig import CondaDependencies
cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_tensorflow_conda_package()
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

'myenv.yml'

In [23]:
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml")

image = Image.create(name = "tf-mnist-image",
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output=True)

Creating image
Running..................................................
Succeeded
Image creation operation finished for image tf-mnist-image:1, operation "Succeeded"


In [28]:
aci_service_name = 'tensorflow-mnist-ayant'

aciconfig = AciWebservice.deploy_configuration(
                cpu_cores = 2, 
                memory_gb = 4)

aci_service = Webservice.deploy_from_image(
                   deployment_config = aciconfig,
                   image = image,
                   name = aci_service_name,
                   workspace = ws)

aci_service.wait_for_deployment(True)

print(aci_service.state)

Running..................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [29]:
print(aci_service.scoring_uri)

http://08409ceb-dc8a-4b9f-9eb4-237642b807dd.westeurope.azurecontainer.io/score
